# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ffbdc572790>
├── 'a' --> tensor([[-0.7532, -0.0333,  0.0205],
│                   [-0.8295,  2.0447, -0.7511]])
└── 'x' --> <FastTreeValue 0x7ffbdc572c70>
    └── 'c' --> tensor([[ 0.0753,  0.1013,  0.6958, -0.6313],
                        [ 0.8173,  0.5237, -0.3259, -0.6396],
                        [ 1.2901,  0.6495, -1.7285,  0.3171]])

In [4]:
t.a

tensor([[-0.7532, -0.0333,  0.0205],
        [-0.8295,  2.0447, -0.7511]])

In [5]:
%timeit t.a

60.4 ns ± 0.11 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ffbdc572790>
├── 'a' --> tensor([[-1.2608,  1.6905, -0.5080],
│                   [ 0.8241,  0.6531,  1.4258]])
└── 'x' --> <FastTreeValue 0x7ffbdc572c70>
    └── 'c' --> tensor([[ 0.0753,  0.1013,  0.6958, -0.6313],
                        [ 0.8173,  0.5237, -0.3259, -0.6396],
                        [ 1.2901,  0.6495, -1.7285,  0.3171]])

In [7]:
%timeit t.a = new_value

66.3 ns ± 0.0548 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7532, -0.0333,  0.0205],
               [-0.8295,  2.0447, -0.7511]]),
    x: Batch(
           c: tensor([[ 0.0753,  0.1013,  0.6958, -0.6313],
                      [ 0.8173,  0.5237, -0.3259, -0.6396],
                      [ 1.2901,  0.6495, -1.7285,  0.3171]]),
       ),
)

In [10]:
b.a

tensor([[-0.7532, -0.0333,  0.0205],
        [-0.8295,  2.0447, -0.7511]])

In [11]:
%timeit b.a

57.4 ns ± 0.0365 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5975, -0.5562,  0.6968],
               [-1.4141,  0.7661,  0.1671]]),
    x: Batch(
           c: tensor([[ 0.0753,  0.1013,  0.6958, -0.6313],
                      [ 0.8173,  0.5237, -0.3259, -0.6396],
                      [ 1.2901,  0.6495, -1.7285,  0.3171]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.0961 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

836 ns ± 1.04 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 45.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 551 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 358 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ffb10897160>
├── 'a' --> tensor([[[-0.7532, -0.0333,  0.0205],
│                    [-0.8295,  2.0447, -0.7511]],
│           
│                   [[-0.7532, -0.0333,  0.0205],
│                    [-0.8295,  2.0447, -0.7511]],
│           
│                   [[-0.7532, -0.0333,  0.0205],
│                    [-0.8295,  2.0447, -0.7511]],
│           
│                   [[-0.7532, -0.0333,  0.0205],
│                    [-0.8295,  2.0447, -0.7511]],
│           
│                   [[-0.7532, -0.0333,  0.0205],
│                    [-0.8295,  2.0447, -0.7511]],
│           
│                   [[-0.7532, -0.0333,  0.0205],
│                    [-0.8295,  2.0447, -0.7511]],
│           
│                   [[-0.7532, -0.0333,  0.0205],
│                    [-0.8295,  2.0447, -0.7511]],
│           
│                   [[-0.7532, -0.0333,  0.0205],
│                    [-0.8295,  2.0447, -0.7511]]])
└── 'x' --> <FastTreeValue 0x7ffb188e9b80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 69.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ffb18bf4c70>
├── 'a' --> tensor([[-0.7532, -0.0333,  0.0205],
│                   [-0.8295,  2.0447, -0.7511],
│                   [-0.7532, -0.0333,  0.0205],
│                   [-0.8295,  2.0447, -0.7511],
│                   [-0.7532, -0.0333,  0.0205],
│                   [-0.8295,  2.0447, -0.7511],
│                   [-0.7532, -0.0333,  0.0205],
│                   [-0.8295,  2.0447, -0.7511],
│                   [-0.7532, -0.0333,  0.0205],
│                   [-0.8295,  2.0447, -0.7511],
│                   [-0.7532, -0.0333,  0.0205],
│                   [-0.8295,  2.0447, -0.7511],
│                   [-0.7532, -0.0333,  0.0205],
│                   [-0.8295,  2.0447, -0.7511],
│                   [-0.7532, -0.0333,  0.0205],
│                   [-0.8295,  2.0447, -0.7511]])
└── 'x' --> <FastTreeValue 0x7ffb0f82ed30>
    └── 'c' --> tensor([[ 0.0753,  0.1013,  0.6958, -0.6313],
                        [ 0.8173,  0.5237, -0.3259, -0.6396],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 47.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.3 µs ± 88.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0753,  0.1013,  0.6958, -0.6313],
                       [ 0.8173,  0.5237, -0.3259, -0.6396],
                       [ 1.2901,  0.6495, -1.7285,  0.3171]],
              
                      [[ 0.0753,  0.1013,  0.6958, -0.6313],
                       [ 0.8173,  0.5237, -0.3259, -0.6396],
                       [ 1.2901,  0.6495, -1.7285,  0.3171]],
              
                      [[ 0.0753,  0.1013,  0.6958, -0.6313],
                       [ 0.8173,  0.5237, -0.3259, -0.6396],
                       [ 1.2901,  0.6495, -1.7285,  0.3171]],
              
                      [[ 0.0753,  0.1013,  0.6958, -0.6313],
                       [ 0.8173,  0.5237, -0.3259, -0.6396],
                       [ 1.2901,  0.6495, -1.7285,  0.3171]],
              
                      [[ 0.0753,  0.1013,  0.6958, -0.6313],
                       [ 0.8173,  0.5237, -0.3259, -0.6396],
                       [ 1.2901,  0.6495, -1.7285,  0.3171]],

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 200 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0753,  0.1013,  0.6958, -0.6313],
                      [ 0.8173,  0.5237, -0.3259, -0.6396],
                      [ 1.2901,  0.6495, -1.7285,  0.3171],
                      [ 0.0753,  0.1013,  0.6958, -0.6313],
                      [ 0.8173,  0.5237, -0.3259, -0.6396],
                      [ 1.2901,  0.6495, -1.7285,  0.3171],
                      [ 0.0753,  0.1013,  0.6958, -0.6313],
                      [ 0.8173,  0.5237, -0.3259, -0.6396],
                      [ 1.2901,  0.6495, -1.7285,  0.3171],
                      [ 0.0753,  0.1013,  0.6958, -0.6313],
                      [ 0.8173,  0.5237, -0.3259, -0.6396],
                      [ 1.2901,  0.6495, -1.7285,  0.3171],
                      [ 0.0753,  0.1013,  0.6958, -0.6313],
                      [ 0.8173,  0.5237, -0.3259, -0.6396],
                      [ 1.2901,  0.6495, -1.7285,  0.3171],
                      [ 0.0753,  0.1013,  0.6958, -0.6313],
                   

In [28]:
%timeit Batch.cat(batches)

146 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

322 µs ± 863 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
